<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Creating Virtual Machines with Python

## Azure Actions Covered

* Creating a virtual network
* Creating a subnet
* Creating a public IP address
* Creating a network interface client (NIC)
* Creating a virtual machine

In this lecture, we're going to take a look at how to create virtual machines via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add some new imports as well.

In [1]:
from azure.identity import AzureCliCredential
# New imports import the Compute and Network management clients
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We'll instantiate our credential as well as our network client. We'll need the compute client later to create the virtual machine, so we can instantiate it here as well.

In [2]:
credential = AzureCliCredential()
compute_client = ComputeManagementClient(credential, AZURE_SUBSCRIPTION_ID)
network_client = NetworkManagementClient(credential, AZURE_SUBSCRIPTION_ID)

We'll store constants that need to be used at various points in setting up the VMs, like we saw in the previous lecture on the Azure CLI. These include:
* Virtual net name
* Subnet name
* Public IP name
* Network interface client (NIC) name
* Virtual machine name

We'll instantiate some of these names when we first use them, so let's just set the virtual net and subnet names.

In [3]:
VNET_NAME = 'test-vnet'
SUBNET_NAME = 'test-subnet'

As in the CLI lecture, we have to first create a virtual network using the `begin_create_or_update()` method. For full list of parameters, see [VirtualNetwork class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-network/azure.mgmt.network.v2022_05_01.models.virtualnetwork?view=azure-python).

In [5]:
poller = network_client.virtual_networks.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    virtual_network_name=VNET_NAME,
    parameters={
        'location': DEFAULT_LOCATION,
        'address_space': {
            'address_prefixes': ['10.0.0.0/16'],
        }
    }
)
result = poller.result()

Let's check that our virtual network has been created successfullly.

In [13]:
result.provisioning_state

'Succeeded'

Next, we need to create the subnet using a similar `begin_create_or_update()` method. For the full list of subnet parameters, see [Subnet class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-network/azure.mgmt.network.v2022_05_01.models.subnet?view=azure-python).

In [14]:
poller = network_client.subnets.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    virtual_network_name=VNET_NAME,
    subnet_name=SUBNET_NAME,
    subnet_parameters={
        'address_prefix': '10.0.0.0/24'
    }
)
subnet_result = poller.result()

We can use the `provisioning_state` attribute to check the subnet was created succesfully.

In [15]:
subnet_result.provisioning_state

'Succeeded'

Once the virtual network and subnet have been created, we can create the public IP address. Let's store the name in a variable, because we'll need it later. You can specify public IP address parameters such as:
* `location` - Azure location for the public IP address
* `sku` - SKU name for public IP address
* `public_ip_allocation_method` - Can be `Static` or `Dynamic`
* `public_ip_address_version` - Can be `IPV4` or `IPV6`

For the full list of parameters, see [PublicIPAddress class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-network/azure.mgmt.network.v2022_05_01.models.publicipaddress?view=azure-python).

In [16]:
IP_NAME = 'vm-test-ip'
poller = network_client.public_ip_addresses.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    public_ip_address_name=IP_NAME,
    parameters={
        'location': DEFAULT_LOCATION,
        'sku': {'name': 'Standard'},
        'public_ip_allocation_method': 'Static',
        'public_ip_address_version': 'IPV4'
    }
)
ip_address_result = poller.result()

Once the poller returns the result, we can verify the name as well as the public IP address that was created.

In [17]:
print(f'Public IP address name: {ip_address_result.name}')
print(f'Public IP address: {ip_address_result.ip_address}')

'vm-test-ip'

The last thing to do before we can create our VM is create the network interface client (NIC). In the parameters, you can specify:
* `location` - Azure location for NIC
* `ip_configurations` - Specify configurations for NIC, such as:
    * `name` - Name for the IP configuration.
    * `subnet` - Information on the relevant subnet, which can be pulled from the returned subnet object.
    * `public_ip_address` - Information on the relevant public IP address, which can be pulled from the returned IP address object.
    
For the full list of parameters, see [NetworkInterface class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-network/azure.mgmt.network.v2022_05_01.models.networkinterface?view=azure-python).

In [22]:
NIC_NAME = 'test-nic'
poller = network_client.network_interfaces.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    network_interface_name=NIC_NAME,
    parameters={
        'location': DEFAULT_LOCATION,
        'ip_configurations': [
            {
                'name': 'test-ip-config',
                'subnet': {'id': subnet_result.id},
                'public_ip_address': {'id': ip_address_result.id}
            }
        ]
    }
)
nic_result = poller.result()

Again, we can verify the creation of the NIC from the returned object.

In [23]:
nic_result.name

'test-nic'

Finally, we can use our `ComputeManagementClient` to create the virtual machine. The `parameters` section is where we can specify various parts of our VM, including:
* `storage_profile` - Section for specifying storage parameters of the VM. For full list of parameters, see [StorageProfile class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_08_01.models.storageprofile?view=azure-python)
    * `image_reference` - Section for specifying VM image parameters. For full list, see [ImageReference class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_08_01.models.imagereference?view=azure-python)
* `hardware_profile` - Section for specifying hardware parameters. For full list of parameters, see [HardwareProfile class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_08_01.models.hardwareprofile?view=azure-python)
    * `vm_size` - Size of VM to create
* `os_profile` - Section for specifying OS options (**note:** these are for administration, `storage_profile` is for setting the name, version, etc. of the OS). For full list of parameters, see [OSProfile class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_08_01.models.osprofile?view=azure-python)
    * `computer_name` - Host name for the OS
    * `admin_username` - User name of the admin user
    * `admin_password` - Password for the admin user
* `network_profile` - This is where you can include information on the NIC to create the virtual machine. For a full list of parameters, see [NetworkProfile class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_08_01.models.networkprofile?view=azure-python)

For the full list of VM parameters, see the [VirtualMachine class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_08_01.models.virtualmachine?view=azure-python)

In [29]:
VM_NAME = 'test-vm'
poller = compute_client.virtual_machines.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    vm_name=VM_NAME,
    parameters={
        'location': DEFAULT_LOCATION,
        # Storage profile
        'storage_profile': {
            'image_reference': {
                'publisher': 'Canonical',
                'offer': 'UbuntuServer',
                'sku': '18.04-LTS',
                'version': 'latest'
            }
        },
        # Hardware profile
        'hardware_profile': {
            'vm_size': 'Standard_A2_v2'
        },
        # Operating system profile
        'os_profile': {
            'computer_name': VM_NAME,
            'admin_username': 'testuser',
            'admin_password': 'testpassword123!'
        },
        # Network profile
        'network_profile': {
            'network_interfaces': [
                {'id': nic_result.id}
            ]
        }
    }
)
vm_result = poller.result()

Let's verify the VM was created and see some of its attributes.

In [30]:
vm_result.name

'test-vm'

Finally, we can list all of our virtual machines to verify the creation using a different method.

In [32]:
for vm in compute_client.virtual_machines.list_all():
    print(vm.name)

test-vm
